## 📚 Exercise 2: Inverted Files

In this exercise, we will understand the functioning of the Inverted Files and how implement them using MapReduce.

### Overview

We learnt in the lecture that terms are typically stored in an inverted list. Now, in the inverted list, instead of only storing document identifiers of the documents in which the term appears, assume we also store an *offset* of the appearance of a term in a document. An $offset$ of a term $l_k$ given a document is defined as the number of words between the start of the document and $l_k$. Thus our inverted list is now:

$l_k= \langle f_k: \{d_{i_1} \rightarrow [o_1,\ldots,o_{n_{i_1}}]\}, 
\{d_{i_2} \rightarrow [o_1,\ldots,o_{n_{i_2}}]\}, \ldots, 
\{d_{i_k} \rightarrow [o_1,\ldots,o_{n_{i_k}}]\} \rangle$

This means that in document $d_{i_1}$ term $l_k$ appears $n_{i_1}$ times and at offset $[o_1,\ldots,o_{n_{i_1}}]$, where $[o_1,\ldots,o_{n_{i_1}}]$ are sorted in ascending order, these type of indices are also known as term-offset indices. An example of a term-offset index is as follows:

**Obama** = $⟨4 : {1 → [3]},{2 → [6]},{3 → [2,17]},{4 → [1]}⟩$

**Governor** = $⟨2 : {4 → [3]}, {7 → [14]}⟩$

**Election** = $⟨4 : {1 → [1]},{2 → [1,21]},{3 → [3]},{5 → [16,22,51]}⟩$

Which is to say that the term **Governor** appear in 2 documents. In document 4 at offset 3, in document 7 at offset 14. Now let us consider the *SLOP/x* operator in text retrieval. This operator has the syntax: *QueryTerm1 SLOP/x QueryTerm2* finds occurrences of *QueryTerm1* within $x$ (but not necessarily in that order) words of *QueryTerm2*, where $x$ is a positive integer argument ($x \geq 1$). Thus $x = 1$ demands that *QueryTerm1* be adjacent to *QueryTerm2*.

### Questions

1. List the set of documents that satisfy the query **Obama** *SLOP/2* **Election**.
2. List each set of values for which the query **Obama** *SLOP/x* **Election** has a different set of documents as answers (starting from $x = 1$). 
3. Consider the general procedure for "merging" two term-offset inverted lists for a given document, to determine where the document satisfies a *SLOP/x* clause (since in general there will be many offsets at which each term occurs in a document). Let $L$ denote the total number of occurrences of the two terms in the document. Assume we have a pointer to the list of occurrences of each term and can move the pointer along this list. As we do so we check whether we have a hit for $SLOP/x$ (i.e. the $SLOP/x$ clause is satisfied). Each move of either pointer counts as a step. Based on this assumption is there a general "merging" procedure to determine whether the document satisfies a $SLOP/x$ clause, for which the following is true? Justify your answer.

    1. The merge can be accomplished in a number of steps linear in $L$ regardless of $x$, and we can ensure that each pointer moves only to the right (i.e. forward).
    2. The merge can be accomplished in a number of steps linear in $L$, but a pointer may be forced to move to the left (i.e. backwards).
    3. The merge can require $x \times L$ steps in some cases.

## Create inverted files using MapReduce
In this exercise, we want to create the inverted files using the MapReduce framework. Write the **map** and **reduce** function.
Note that the code of **run_mapreduce** is just a simulation of the behavior of  map-reduce system.

## Goal:
- Create inverted files using the MapReduce framework.  
- Test the implementation on `epfldocs.txt` data.

### What you are learning in this exercise:
- Having a better understanding of inverted files.
- Learn how to implement inverted files using the MapReduce framework. 

In [3]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')

stemmer = PorterStemmer()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\fosmo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:

# Tokenize, stem a document
def tokenize(text):
    """
    It tokenizes and stems an input text.
    
    :param text: str, with the input text
    :return: list, of the tokenized and stemmed tokens.
    """
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]
docs_n_doc_ids = list(zip(range(len(documents)), documents))

UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in position 135: character maps to <undefined>

In [5]:
# Take a pair (doc_id, content). Example: (0, ['how', 'to', 'bake', 'bread', 'without', 'bake', 'recip'])
# Return: list of (word, doc_id). Example: ('bake', 0)
# For simplicity, we assume each mapper handles a document
def map_function(doc_id, content):
    keyvals = []
    for word in content:
        keyvals.append((word,doc_id)) #attention doc id
    return keyvals

In [6]:
# Take a list of pairs (word, doc_id). Example:  [('bake', 0), ('bake', 0), ('bake', 3)]
# Return: (word, frequency, list of document ids). Example ('bake', 3, [0, 0, 3])
# For simplicity, we assume each reducer handles a word
def reduce_function(lst):
    total = 0
    word = lst[0][0]
    doc_ids = []
    for w, doc_id in lst:
        assert(word==w) # Assume each reducer handles 1 word
        total=total+1
        doc_ids.append(doc_id)
    return (word, total, doc_ids)

In [7]:
# We simulate the mapreduce framework by this function
def run_mapreduce(docs_n_doc_ids):
    # Map phase
    key_values = []
    for doc_id, doc_content in docs_n_doc_ids:
        key_values.extend(map_function(doc_id, doc_content))
    # Shuffle phase
    values = set(map(lambda x:x[0], key_values))
    grouped_key_values = [[y for y in key_values if y[0]==x] for x in values]
    # Reduce phase
    inverted_files = []
    for grouped_key_value in grouped_key_values:
        word, total, doc_ids = reduce_function(grouped_key_value)
        inverted_files.append((word, total, doc_ids))
    return inverted_files

In [8]:
run_mapreduce(docs_n_doc_ids)

NameError: name 'docs_n_doc_ids' is not defined